## Buildkite Demo

### Prequesites

In [62]:
import os
import time

PERSONAL_ACCESS_TOKEN: str = os.environ.get("BUILDKITE_ACCESS_TOKEN")
ORGANISATION: str = "bmwgroup"

### Auth

In [63]:
from pybuildkite.buildkite import Buildkite, BuildState

buildkite: Buildkite = Buildkite()
buildkite.set_access_token(PERSONAL_ACCESS_TOKEN)

### Organisation

In [64]:
from typing import Any, Dict

organisation_infos: Dict[str, Any] = buildkite.organizations().get_org(ORGANISATION)
print(organisation_infos)

{'id': '0192f6f2-2d4d-4127-acf6-877aeaf204a0', 'graphql_id': 'T3JnYW5pemF0aW9uLS0tMDE5MmY2ZjItMmQ0ZC00MTI3LWFjZjYtODc3YWVhZjIwNGEw', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup', 'web_url': 'https://buildkite.com/bmwgroup', 'name': 'bmwgroup', 'slug': 'bmwgroup', 'agents_url': 'https://api.buildkite.com/v2/organizations/bmwgroup/agents', 'emojis_url': 'https://api.buildkite.com/v2/organizations/bmwgroup/emojis', 'created_at': '2024-11-04T11:31:59.436Z', 'pipelines_url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines'}


### Pipelines

In [65]:
from multiprocessing.connection import Client
from pybuildkite.pipelines import Pipelines

pipelines: Pipelines = buildkite.pipelines()

print(pipelines)

pipeline_path: str = pipelines.path
pipeline_client: Client = pipelines.client

print(f"Path: {pipeline_path}\nClient: {pipeline_client}")


Path: https://api.buildkite.com/v2/organizations/{}/pipelines/
Client: <pybuildkite.client.Client object at 0x7dc6c1f2e2d0>


### Build Info

In [66]:
from pybuildkite.builds import Builds

pipeline: str = "bazel-build"

state_running: BuildState = BuildState.RUNNING
scheduled: BuildState = BuildState.SCHEDULED
canceled: BuildState = BuildState.CANCELED
failed: BuildState = BuildState.FAILED
passed: BuildState = BuildState.PASSED

running_or_scheduled: Builds = buildkite.builds().list_all_for_pipeline(ORGANISATION, pipeline, states=[state_running, scheduled])
print(f"Running or scheduled builds: {[build["message"] for build in running_or_scheduled]}")

canceled_or_failed: Builds = buildkite.builds().list_all_for_pipeline(ORGANISATION, pipeline, states=[canceled, failed])
print(f"Canceled or failed builds: {[build["message"] for build in canceled_or_failed]}")

passed_builds: Builds = buildkite.builds().list_all_for_pipeline(ORGANISATION, pipeline, states=[passed])
print(f"Canceled or passed builds: {[build["message"] for build in passed_builds]}")

Running or scheduled builds: []
Canceled or failed builds: ['Build via Python Client!', 'Make script executable', 'Update pipeline.yml', 'Add bazel build to pipeline', 'Run only bazel install', 'Add testengine pipeline configuration', 'Trying a dumb new file', 'Test multiple commands in single step', 'Test multiple commands in single step', 'Test multiple commands in single step', 'Test multiple commands in single step', 'fix bazel test', 'add bazel test //...', 'fix bazel build command', 'fix auto retry', 'fix auto retry', 'add auto retry', 'add depends on', "Merge branch 'bazelbuild:master' into master", 'sqash steps', 'add bazel build', 'test install bazelisk v2', 'test install bazelisk v2', 'Add bazelisk installation script', 'test-bazel-build-again', 'test-bazel-build', 'test-bazel-build']
Canceled or passed builds: ['Build via Python Client!', 'Merge pull request #2 from buildkite-demo-org/simonrummert-patch-1\n\nRemove install step and test step', 'Remove install step', 'Remove 

### Create Build

In [67]:
pipeline: str = "bazel-build"

build_info = buildkite.builds().create_build(ORGANISATION, pipeline, 'HEAD', 'master',
                                             clean_checkout=True, message="Build via Python Client!")

print(f"Build sent!\nBuild Info:{build_info}")
CREATED_AT = build_info["created_at"]

Build sent!
Build Info:{'id': '01936d5a-060a-4188-9bc6-72ac24d1b74e', 'graphql_id': 'QnVpbGQtLS0wMTkzNmQ1YS0wNjBhLTQxODgtOWJjNi03MmFjMjRkMWI3NGU=', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines/bazel-build/builds/37', 'web_url': 'https://buildkite.com/bmwgroup/bazel-build/builds/37', 'number': 37, 'state': 'scheduled', 'cancel_reason': None, 'blocked': False, 'blocked_state': '', 'message': 'Build via Python Client!', 'commit': 'HEAD', 'branch': 'master', 'tag': None, 'env': {}, 'source': 'api', 'author': None, 'creator': {'id': '0193004b-dc06-4728-9a4f-d8e37ca04f09', 'graphql_id': 'VXNlci0tLTAxOTMwMDRiLWRjMDYtNDcyOC05YTRmLWQ4ZTM3Y2EwNGYwOQ==', 'name': 'Oliver Königsberger', 'email': 'oliver@koenigsberger.de', 'avatar_url': 'https://avatars.githubusercontent.com/u/102287892?v=4', 'created_at': '2024-11-06T07:06:31.813Z'}, 'created_at': '2024-11-27T11:20:36.637Z', 'scheduled_at': '2024-11-27T11:20:36.525Z', 'started_at': None, 'finished_at': None, 'meta_data': {}

### Build Response

In [70]:
from typing import Optional
from pybuildkite.client import Response

BUILD_RESULT: Optional[Any] = None

while True:
    builds_response: Response = buildkite.builds().list_all(page=1, with_pagination=True)

    while builds_response.next_page:
        print("Searching for build result")
        for build in builds_response.body:
            if build["created_at"] == CREATED_AT:
                BUILD_RESULT = build
                break

        builds_response = buildkite.builds().list_all(page=builds_response.next_page, with_pagination=True)

    if BUILD_RESULT is not None and BUILD_RESULT["state"] is not "SCHEDULED":
        break
    else:
        time.sleep(1)

print(f"Build: {BUILD_RESULT["message"]} was {BUILD_RESULT["state"]}")
print(BUILD_RESULT)


<>:18: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
<>:18: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
/tmp/ipykernel_1585164/3055293017.py:18: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
  if BUILD_RESULT is not None and BUILD_RESULT["state"] is not "SCHEDULED":


Searching for build result
Searching for build result
Build: Build via Python Client! was passed
{'id': '01936d5a-060a-4188-9bc6-72ac24d1b74e', 'graphql_id': 'QnVpbGQtLS0wMTkzNmQ1YS0wNjBhLTQxODgtOWJjNi03MmFjMjRkMWI3NGU=', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines/bazel-build/builds/37', 'web_url': 'https://buildkite.com/bmwgroup/bazel-build/builds/37', 'number': 37, 'state': 'passed', 'cancel_reason': None, 'blocked': False, 'blocked_state': '', 'message': 'Build via Python Client!', 'commit': '2aac26f48791596b6c1dae593ef5d22d48d1ea22', 'branch': 'master', 'tag': None, 'env': {}, 'source': 'api', 'author': None, 'creator': {'id': '0193004b-dc06-4728-9a4f-d8e37ca04f09', 'graphql_id': 'VXNlci0tLTAxOTMwMDRiLWRjMDYtNDcyOC05YTRmLWQ4ZTM3Y2EwNGYwOQ==', 'name': 'Oliver Königsberger', 'email': 'oliver@koenigsberger.de', 'avatar_url': 'https://avatars.githubusercontent.com/u/102287892?v=4', 'created_at': '2024-11-06T07:06:31.813Z'}, 'created_at': '2024-11-27T11:20:36.